# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

In [1]:
import string
from pathlib import Path
from typing import Iterable

import nltk
import numpy as np
import pandas as pd
import pymorphy2
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [2]:
DATA_DIR = Path('data/')

## Задачи для совместного разбора

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [31]:
with open(DATA_DIR.joinpath('litw-win.txt'), encoding='cp1251') as f:
    words = [line.strip().split()[-1].lower() for line in f]

words[-5:]

['высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [32]:
text = ('с велечайшим усилием выбравшись из потока убегающих людей Кутузов со '
        'свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий')

In [33]:
def find_replacements(text: str, dictionary: Iterable) -> dict[str, tuple[int, list[str]]]:
    dictionary = set(dictionary)
    replacements = {}
    for word in text.lower().split():
        if word not in dictionary:
            sorted_edit_distances = sorted(
                map(lambda x: (nltk.edit_distance(word, x), x), dictionary),
                key=lambda x: x[0]
            )
            min_edit_distance = sorted_edit_distances[0][0]
            replacement_list = []
            for edit_distance, dict_word in sorted_edit_distances:
                if edit_distance > min_edit_distance:
                    break
                replacement_list.append(dict_word)
            replacements[word] = (min_edit_distance, replacement_list)

    return replacements

In [34]:
replacements = find_replacements(text, words)
replacements

{'велечайшим': (1, ['величайшим']), 'уменьшевшейся': (1, ['уменьшившейся'])}

In [35]:
fixed_text = text
for word, repl in replacements.items():
    fixed_text = fixed_text.replace(word, repl[1][0])
fixed_text

'с величайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [36]:
task1_text = (
    'Считайте слова из файла `litw-win.txt` и запишите их в список `words`. '
    'В заданном предложении исправьте все опечатки, заменив слова с опечатками '
    'на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. '
    'Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.'
)
stemmer = nltk.SnowballStemmer('russian')
lemmer = pymorphy2.MorphAnalyzer()

In [37]:
pd.DataFrame(
    data=[
        [word, stemmer.stem(word), lemmer.parse(word)[0].normal_form]
        for word in nltk.word_tokenize(task1_text)
        if word not in string.punctuation
    ],
    columns=['word', 'stemma', 'lemma']
).sample(5)

,word,stemma,lemma
6,запишите,запиш,записать
30,из,из,из
31,списка,списк,список
5,и,и,и
29,слова,слов,слово


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [38]:
CountVectorizer().fit_transform(nltk.sent_tokenize(task1_text)).toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [3]:
descriptions_df = pd.read_csv(DATA_DIR.joinpath('preprocessed_descriptions.csv'), sep=',')
descriptions_df = descriptions_df.rename(columns={'preprocessed_descriptions': 'description'})
descriptions_df.info()
descriptions_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         30000 non-null  object
 1   description  29369 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


,name,description
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [4]:
descriptions = descriptions_df['description'].dropna()
words = list({word for text in descriptions for word in nltk.word_tokenize(text)})
len(words)

30823

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [41]:
rng = np.random.default_rng()
random_pairs = rng.choice(words, size=(5, 2)).tolist()
list(map(lambda x: (x, nltk.edit_distance(x[0], x[1])), random_pairs))

[(['pharmacists', 'livens'], 10),
 (['tomake', 'tombs'], 3),
 (['warningthis', 'handwith'], 7),
 (['complaints', 'sexymommalucass'], 11),
 (['globs', 'met'], 5)]

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [42]:
def similarity(word: str, words: list[str], k: int) -> list[str]:
    return sorted(words, key=lambda x: nltk.edit_distance(word, x))[:k]

In [43]:
similarity('hello', words, 5)

['hello', 'mello', 'jello', 'hell', 'help']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для лемматизации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [44]:
stemmer = nltk.SnowballStemmer('english')
lemmatizer = nltk.WordNetLemmatizer()

In [45]:
df = pd.DataFrame(
    data={
        'stemmed_word': map(stemmer.stem, words),
        'normalized_word': map(lemmatizer.lemmatize, words),
    },
    index=pd.Series(words, name='words')
)
df.head()

,stemmed_word,normalized_word
words,,
mad,mad,mad
sunmaid,sunmaid,sunmaid
inspire,inspir,inspire
granulated,granul,granulated
muschelessen,muschelessen,muschelessen


In [46]:
print(f'Всего слов: {len(df)}')
print(f'Основ отличается: {len(set(df.index) - set(df["stemmed_word"]))}')
print(f'Лемм отличается: {len(set(df.index) - set(df["normalized_word"]))}')

Всего слов: 30823
Основ отличается: 15330
Лемм отличается: 3131


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [47]:
stop_words = set(stopwords.words('english'))

In [48]:
all_words = [word for text in descriptions for word in nltk.word_tokenize(text)]
words_without_stopwords = [word for word in all_words if word not in stop_words]

In [49]:
n_words = len(all_words)
n_words_without_stopwords = len(words_without_stopwords)
print(f'Слов всего: {n_words}')
print(f'Слов всего после удаления стоп-слов: {n_words_without_stopwords}')
print(f'Доля стоп-слов {(n_words - n_words_without_stopwords) / n_words:.2%}')

Слов всего: 1071865
Слов всего после удаления стоп-слов: 581919
Доля стоп-слов 45.71%


In [50]:
print(f'Топ-10 по частоте (до): \n{nltk.FreqDist(all_words).most_common(10)}')
print(f'Топ-10 по частоте (после): \n{nltk.FreqDist(words_without_stopwords).most_common(10)}')

Топ-10 по частоте (до): 
[('the', 40210), ('a', 34994), ('and', 30279), ('this', 27048), ('i', 25111), ('to', 23499), ('is', 20290), ('it', 19863), ('of', 18372), ('for', 15988)]
Топ-10 по частоте (после): 
[('recipe', 14957), ('make', 6353), ('time', 5180), ('use', 4635), ('great', 4453), ('like', 4175), ('easy', 4175), ('one', 3886), ('good', 3820), ('made', 3814)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [16]:
descriptions_sample = descriptions_df.dropna().sample(5)
descriptions_sample

,name,description
10598,eric s stuff,i was talking with a friend at work the other ...
755,anne s poppy seed dressing for fruit salad,this dressing spruces up a fruit salad for a s...
10027,easy granola bars,this is an easy granola bar recipe they are mu...
28650,vegetable crudites with greek yogurt dip,this is sanjeev kapoors recipe from the young ...
22275,really easy and good coconut cream pie,just like the heading sayssimple and easy and ...


In [17]:
tfid_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfid_vectorizer.fit(descriptions_sample['description'])
descriptions_sample['vector'] = descriptions_sample['description'].apply(
    lambda x: tfid_vectorizer.transform([x]).toarray()
)
print(descriptions_sample.to_string())

                                             name                                                                                                                                                                                                                                                                                                                                                                                                    description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [21]:
name_values = [name[:8] + '...' for name in descriptions_sample['name'].array]
vector_values = [v.reshape(-1) for v in descriptions_sample['vector'].array]

In [22]:
cosine_distances_df = pd.DataFrame(
    data=cosine_distances(vector_values, vector_values),
    index=name_values,
    columns=name_values,
)
cosine_distances_df

,eric s ...,anne s p...,easy gra...,vegetabl...,really e...
eric s ...,0.000000,1.0,1.000000,0.974138,0.812319
anne s p...,1.000000,0.0,1.000000,1.000000,1.000000
easy gra...,1.000000,1.0,0.000000,0.935780,0.883489
vegetabl...,0.974138,1.0,0.935780,0.000000,1.000000
really e...,0.812319,1.0,0.883489,1.000000,0.000000


In [23]:
cosine_similarity_df = pd.DataFrame(
    data=cosine_similarity(vector_values, vector_values),
    index=name_values,
    columns=name_values,
)
cosine_similarity_df

,eric s ...,anne s p...,easy gra...,vegetabl...,really e...
eric s ...,1.000000,0.0,0.000000,0.025862,0.187681
anne s p...,0.000000,1.0,0.000000,0.000000,0.000000
easy gra...,0.000000,0.0,1.000000,0.064220,0.116511
vegetabl...,0.025862,0.0,0.064220,1.000000,0.000000
really e...,0.187681,0.0,0.116511,0.000000,1.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [24]:
eps = 1e-6
similarity_top = cosine_similarity_df.to_numpy()
np.sort(similarity_top[(0 + eps < similarity_top) & (similarity_top < 1 - eps)])[::-1]

array([0.18768099, 0.18768099, 0.11651141, 0.11651141, 0.06422045,
       0.06422045, 0.02586218, 0.02586218])

In [57]:
# Два различных рецепта тем более похожи,
# чем меньше косинусное расстояние между ними
# (или чем больше косинусная схожесть: 1 - cosine)
